In [10]:
import numpy as np

In [11]:
#operations
def combo(operand,register):
    if operand<=3:
        return operand
    elif operand<7:
        return register[operand-4]
    else:
        raise ValueError('Illegal combo operand 7')

def adv(operand,register):
    register[0]= int(register[0]//(2**combo(operand,register)))
    return -1,''

def bxl(operand,register):
    register[1]^= operand
    return -1,''

def bst(operand,register):
    register[1]=combo(operand,register)%8
    return -1,''

def jnz(operand,register):
    if register[0]!=0:
        return operand,''
    else:
        return -1,''

def bxc(operand,register):
    register[1]^=register[2]
    return -1,''

def out(operand,register):
    #print (f'{combo(operand,register)%8},',end='')
    return -1,str(combo(operand,register)%8)

def bdv(operand,register):
    register[1]= int(register[0]//(2**combo(operand,register)))
    return -1,''

def cdv(operand,register):
    register[2]= int(register[0]//(2**combo(operand,register)))
    return -1,''

instr=(adv,bxl,bst,jnz,bxc,out,bdv,cdv)


In [12]:
register=np.zeros(3,dtype=int)
with open('input_17.txt') as f:
    for line in f:
        if len(line)<2:
            continue
        elif line[9]=='A':
            register[0]=int(line[11:].strip())
        elif line[9]=='B':
            register[1]=int(line[11:].strip())
        elif line[9]=='C':
            register[2]=int(line[11:].strip())
        else:
            code=np.array(list(map(int,line[9:].strip().split(','))))
print (register)
print (len(code),code)
regsave=register.copy()

[30118712        0        0]
16 [2 4 1 3 7 5 4 2 0 3 1 5 5 5 3 0]


In [13]:
pointer=0
while pointer<len(code):
    ret,string=instr[code[pointer]](code[pointer+1],register)
    #print('Instruction',code[pointer],'Op',code[pointer+1],'Return',ret)
    if ret==-1:
        pointer +=2
    else:
        pointer = ret

In [14]:
print('code',code)
print('code[-2:]',code[-2:])
print('code[-2::-1]',code[-2::-1])
print('code[-1:-3:-1]',code[-1:-3:-1])

code [2 4 1 3 7 5 4 2 0 3 1 5 5 5 3 0]
code[-2:] [3 0]
code[-2::-1] [3 5 5 5 1 3 0 2 4 5 7 3 1 4 2]
code[-1:-3:-1] [0 3]


In [15]:
base = 0
found=[0]
for length in range(1,len(code)+1):
    nextlev=[]
    for seed in found:
        test=seed
        print('Length',length,'Base',seed)
        for count in range(8):
            register=regsave.copy()
            register[0]=test
            pointer=0
            outind=0
            correct=1
            #print('Test A',register)
            allout=[]
            while pointer<len(code): #and correct:
                ret,output=instr[code[pointer]](code[pointer+1],register)
                if len(output)>0:
                #print('Output',output,'Code at ',outind,':',code[outind])
                #if code[outind]!=int(output):
                #    print('Test A failed',test)
                #    correct=0
                    outind+=1
                    allout.append(int(output))
                #if outind>len(code):
                #    correct=0
                if ret==-1:
                    pointer +=2
                else:
                    pointer = ret  
            if all(code[-length:]==np.array(allout)):
                print('Test A successful',test,code[-length:],allout)
                nextlev.append(test*8)
            else: 
                print('Test A unsuccessful',test,code[-length:],allout)
            test+=1
    found=np.array(nextlev)    
            

Length 1 Base 0
Test A unsuccessful 0 [0] [6]
Test A unsuccessful 1 [0] [7]
Test A unsuccessful 2 [0] [5]
Test A unsuccessful 3 [0] [6]
Test A unsuccessful 4 [0] [2]
Test A unsuccessful 5 [0] [3]
Test A successful 6 [0] [0]
Test A unsuccessful 7 [0] [1]
Length 2 Base 48
Test A unsuccessful 48 [3 0] [0, 0]
Test A successful 49 [3 0] [3, 0]
Test A unsuccessful 50 [3 0] [5, 0]
Test A unsuccessful 51 [3 0] [6, 0]
Test A unsuccessful 52 [3 0] [2, 0]
Test A successful 53 [3 0] [3, 0]
Test A unsuccessful 54 [3 0] [1, 0]
Test A unsuccessful 55 [3 0] [2, 0]
Length 3 Base 392
Test A unsuccessful 392 [5 3 0] [7, 3, 0]
Test A successful 393 [5 3 0] [5, 3, 0]
Test A unsuccessful 394 [5 3 0] [1, 3, 0]
Test A unsuccessful 395 [5 3 0] [6, 3, 0]
Test A unsuccessful 396 [5 3 0] [1, 3, 0]
Test A successful 397 [5 3 0] [5, 3, 0]
Test A unsuccessful 398 [5 3 0] [4, 3, 0]
Test A unsuccessful 399 [5 3 0] [1, 3, 0]
Length 3 Base 424
Test A unsuccessful 424 [5 3 0] [3, 3, 0]
Test A successful 425 [5 3 0] [5, 3

In [20]:
np.min(found//8)

236555995274861

In [ ]:
#Solution from subreddit.
from re import findall
a, b, c, *prog = [int(n) for n in findall("(\d+)", open("input_17.txt").read())]

def run(prog, a):
    ip, b, c, out = 0, 0, 0, []
    while ip>=0 and ip<len(prog):
        lit, combo = prog[ip+1], [0,1,2,3,a,b,c,99999][prog[ip+1]]
        match prog[ip]:
            case 0: a = int(a / 2**combo)       # adv
            case 1: b = b ^ lit                 # bxl
            case 2: b = combo % 8               # bst
            case 3: ip = ip if a==0 else lit-2  # jnz
            case 4: b = b ^ c                   # bxc
            case 5: out.append(combo % 8)       # out
            case 6: b = int(a / 2**combo)       # bdv
            case 7: c = int(a / 2**combo)       # cdv
        ip+=2
    return out
print("Part 1:", ",".join(str(n) for n in run(prog, a)))

target = prog[::-1]
def find_a(a=0, depth=0):
    if depth == len(target):
        return a
    for i in range(8):
        output = run(prog, a*8 + i)
        print(output,i,a*8+i)
        if output and output[0] == target[depth]:
            if result := find_a((a*8 + i), depth+1): 
                return result
    return 0
print("Part 2:", find_a())

Part 1: 1,7,6,5,1,0,5,0,7
[6] 0 0
[7] 1 1
[5] 2 2
[6] 3 3
[2] 4 4
[3] 5 5
[0] 6 6
[0, 0] 0 48
[3, 0] 1 49
[7, 3, 0] 0 392
[5, 3, 0] 1 393
[7, 5, 3, 0] 0 3144
[5, 5, 3, 0] 1 3145
[7, 5, 5, 3, 0] 0 25160
[5, 5, 5, 3, 0] 1 25161
[7, 5, 5, 5, 3, 0] 0 201288
[5, 5, 5, 5, 3, 0] 1 201289
[1, 5, 5, 5, 3, 0] 2 201290
[4, 1, 5, 5, 5, 3, 0] 0 1610320
[3, 1, 5, 5, 5, 3, 0] 1 1610321
[7, 3, 1, 5, 5, 5, 3, 0] 0 12882568
[5, 3, 1, 5, 5, 5, 3, 0] 1 12882569
[1, 3, 1, 5, 5, 5, 3, 0] 2 12882570
[6, 3, 1, 5, 5, 5, 3, 0] 3 12882571
[7, 3, 1, 5, 5, 5, 3, 0] 4 12882572
[1, 3, 1, 5, 5, 5, 3, 0] 5 12882573
[4, 3, 1, 5, 5, 5, 3, 0] 6 12882574
[1, 3, 1, 5, 5, 5, 3, 0] 7 12882575
[5, 1, 5, 5, 5, 3, 0] 2 1610322
[6, 1, 5, 5, 5, 3, 0] 3 1610323
[6, 1, 5, 5, 5, 3, 0] 4 1610324
[2, 1, 5, 5, 5, 3, 0] 5 1610325
[2, 1, 5, 5, 5, 3, 0] 6 1610326
[4, 1, 5, 5, 5, 3, 0] 7 1610327
[6, 5, 5, 5, 3, 0] 3 201291
[6, 5, 5, 5, 3, 0] 4 201292
[2, 5, 5, 5, 3, 0] 5 201293
[2, 5, 5, 5, 3, 0] 6 201294
[5, 5, 5, 5, 3, 0] 7 201295
[1, 5,

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/var/folders/8y/csqgq0y10tg9m73_s1yy_hxh0000gq/T/ipykernel_12160/38314455.py:2: SyntaxWarning: invalid escape sequence '\d'
  a, b, c, *prog = [int(n) for n in findall("(\d+)", open("input_17.txt").read())]
